In [4]:
import nltk 
from nltk.stem import WordNetLemmatizer
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
from getpass import getpass
import re

In [5]:
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
new_model = load_model('chatbotmodel.h5')
index_responses = pickle.load(open('index_responses.pkl', 'rb'))
lemmatizer = pickle.load(open('lemmatizer.pkl', 'rb'))

In [2]:
usernames_emails = {}
usernames_passwords = {}
usernames_secretwords = {}

In [4]:
def prediction(user_input):

    tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
    model = load_model('chatbotmodel.h5')
    index_responses = pickle.load(open('index_responses.pkl', 'rb'))
    stopwords = pickle.load(open('stopwords.pkl', 'rb'))
    lemmatizer = pickle.load(open('lemmatizer.pkl', 'rb'))

    text = user_input.lower()
    text = re.sub("[0-9]","", text)
    text = text.replace('?', '')
    text = text.replace(',', '')
    text = re.sub("  "," ", text)
    
    userTextClean = [] 
    for word in text.split(' '):
        if word not in stopwords:
            userTextClean.append(word)
    
    lemmaText = []
    for word in userTextClean:
        lemmaText.append(lemmatizer.lemmatize(word.strip('"')))

    finalText = ' '.join(lemmaText)
    
    sequenced_input = tokenizer.texts_to_sequences([user_input])
    padded_input = pad_sequences(sequenced_input, truncating='post', maxlen=10)
    result = new_model.predict(padded_input)
    results = {}
    for i in range(len(result[0])):
        results[i] = new_model.predict(padded_input)[0][i]
    sorted_results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=True)}
    list_results = list(sorted_results)
    predicted_class_index = int(list_results[0])
    return predicted_class_index

def get_response(input_class, responses):
    return responses[input_class]


In [9]:
print('HelpBot: Hello. I am HelpBot. What can I help you with?')
while True:
    user_input = input('User: ')
    context = prediction(user_input)
    res = get_response(context, index_responses)
    if user_input == 'quit':
        break
    else:    
        print(f'HelpBot: {res}')
        print(f'HelpBot: Is this what you wanted help with? Type \'yes\' to continue or \'no\' to retype your query')
        is_helpful = input('User: ')
        if is_helpful == 'yes':
            if context == 4:
                print(f'HelpBot: In order to create a new account, we will need your username, first name, last name, and email address.')
                
                username = input('Desired username: ')
                while username in usernames_passwords.keys():
                    print('This username is already taken. Please choose another one.')
                    username = input('Desired username: ')
                else:
                    pass
                
                first_name = input('First name: ')
                last_name = input('Last name: ')
                
                email_address = input('E-mail address: ')
                while email_address in usernames_emails.values():
                    print('This e-mail address is already associated with another account. Please use another one.')
                    email_address = input('E-mail address: ')
                else:
                    pass
                    
                password_1 = getpass(prompt='Password: ')
                password_2 = getpass(prompt='Confirm password: ')
                while password_1 != password_2:
                    print(f'HelpBot: The passwords do not match. Please re-enter your password.')
                    password_1 = getpass(prompt='Password: ')
                    password_2 = getpass(prompt='Confirm password: ')
                else:
                    usernames_passwords[username] = password_1
                    usernames_emails[username] = email_address
                
                print('HelpBot: Please give a secret recovery word in case you lose your password.')
                secret_word = input('Secret word: ')
                usernames_secretwords[username] = secret_word
                print(f'Okay. User \'{username}\' created. You may now log in on www.dummysite.com/login. What else can I help you with?')
                
            # Recover password
            elif context == 8:
                print(f'HelpBot: In order to recover your password, you must provide your username and secret recovery word.')
                username = input('Username: ')
                for i in list(range(3)):
                    if username not in usernames_passwords.keys():
                        username = input('The username does not exist. Enter your username: ')
                        if i == 2:
                            print(f'You entered an invalid username three times. Let\'s start over. What can I help you with?')
                if username in usernames_passwords.keys():
                    for i in list(range(3)):
                        secret_word = input('Secret recovery word: ')
                        if usernames_secretwords[username] == secret_word:
                            print(f'HelpBot: You have successfully entered the secret recovery word to your username. Your password is: \'{usernames_passwords[username]}\'. What else can I help you with?')
                            break
                        elif i == 2:
                            print(f'HelpBot: You have not entered the correct secret recovery word 3 times in a row. Please try again later.\n What else can I help you with?')
                        else:
                            print(f'HelpBot: You have not entered the correct secret recovery word for that username. Try again.')                
                else:
                    continue
            else:
                print('HelpBot: Perfect! What else can I help you with?')
        elif is_helpful != 'yes': 
            print('HelpBot: Please reformulate your query for a better search result.')

else:
    pass

HelpBot: Hello. I am HelpBot. What can I help you with?
User: recover password
HelpBot: You asked to recover your password.
HelpBot: Is this what you wanted help with? Type 'yes' to continue or 'no' to retype your query
User: yes
HelpBot: In order to recover your password, you must provide your username and secret recovery word.
Username: horseman2
The username does not exist. Enter your username: rice
The username does not exist. Enter your username: rice
The username does not exist. Enter your username: rice
You entered an invalid username three times. Let's start over. What can I help you with?
User: create an account
HelpBot: You asked to create a new account.
HelpBot: Is this what you wanted help with? Type 'yes' to continue or 'no' to retype your query
User: yes
HelpBot: In order to create a new account, we will need your username, first name, last name, and email address.
Desired username: student007
First name: james
Last name: bond
E-mail address: jamesbond@mail.com
Password: 

KeyboardInterrupt: Interrupted by user

In [6]:
usernames_passwords, usernames_emails,usernames_secretwords

index_responses

{0: 'To change your order, please log in and visit www.dummysite.com/change_order. These are the elligibility conditions: \n 1) ... \n 2) ... \n etc.',
 1: 'To check your invoices, please log in and visit www.dummysite.com/check_invoices. You have access to the previous 24 detailed invoices.',
 2: 'To contact customer service, you can call 1-800-555-1234 from Monday to Friday, 9:00AM - 5:00PM EST.',
 3: 'To contact a human agent, you can call 1-800-555-1234 from Monday to Friday, 9:00AM - 5:00PM EST.',
 4: 'You asked to create a new account.',
 5: 'To get an invoice, please have your personal information ready along with your order details and call 1-800-555-1234',
 6: 'To get a refund, please log in and visit www.dummysite.com/refund. You will be asked to provide the reason of your refund.',
 7: 'If you are having a payment issue, please contact our billing department at 1-800-555-1234.',
 8: 'You asked to recover your password.',
 9: 'You can track your packages on https://www.purola